# Functional Python Practice

This notebook is a series of exercises designed to give you practice with the concepts of functional programming in Python.

Like the testing PA, this assignment lends itself to very short answers, and using Generative AI defeats the purpose. 

**The use of Gen AI is prohibted.**

Make a best effort attempt at each of these, it is OK for a few to be imperfect. I am glad to share solutions after the assignment is turned in.

Each assignment has a few non-comprehensive test cases.
For extra practice, see if you can find an edge case or two where you pass the tests but still have issues in your solution.

Onurcan Buyukkalkan

In [59]:
from typing import Iterable, Generator
import string
import functools

## 1. filter

Use `filter` to complete this function. You may write a helper function, or use a lambda.

**You may not use a loop.**


In [60]:
def longer_than(words: str, n: int) -> Iterable[str]:
    """ returns iterable of words longer than n characters """
    return (filter(lambda word: len(word) > n, words))

In [61]:
animals = ["ape", "bear", "boar", "tiger", "zebra", "hippopotamus", "cat"]
result = list(longer_than(animals, 5))
assert result == ["hippopotamus"], f"bad response: {result}"

result = list(longer_than(animals, 0))
assert result == animals, f"bad response: {result}"

result = list(longer_than(animals, 100))
assert result == [], f"bad response: {result}"

print("Success!")

Success!


## 2. map

Use `map` to complete this function. 

**You may not use a loop.**


For an extra challenge, you can implement that helper function without using a loop as well.

In [62]:

def remove_punctuation(word: str) -> str:
    return "".join(ch for ch in word if ch not in string.punctuation)
    
def without_punctuation(words: list[str]) -> Iterable[str]:
    """ returns words with all punctuation stripped out """
    return (map(remove_punctuation, words))

In [63]:
result = list(without_punctuation(["abc"])) 
assert result == ["abc"], f"{result}"

result = list(without_punctuation(["a.b.c"]))
assert result == ["abc"], f"{result}"

result = list(without_punctuation(["a.b.c", "def?", "g#h!i?"])) 
assert result == ["abc", "def", "ghi"], f"{result}"

print("Success!!")

Success!!


## 3. partial

Take a look at the helper function `remove_punctuation` -- it seems like it'd make sense to have a function `remove_chars(s: str, chars: str) -> str` that is more flexible.

But doing so would mean that the function is no longer `f(str) -> str` and not suitable for use with `map`!

This is a perfect application of `partial`, below please do the following:

1. Implement `remove_chars`
2. Use `remove_chars` with `functools.partial` to implement `without_chars

In [64]:
def remove_chars(word: str, chars: str) -> str:
    """
    Return a new string that is the original string "s" without any characters that appear in "chars"
    """
    return "".join(ch for ch in word if ch not in chars)

def without_chars(words: list[str], chars: str) -> Iterable[str]:
    """ returns words with all punctuation stripped out """
    no_punctuation = functools.partial(remove_chars, chars=chars)
    return (map(no_punctuation, words))

In [65]:
result = list(without_chars(["abc"], "a")) 
assert result == ["bc"], f"bad result: {result}"

result = list(without_chars(["abc", "banana"], "ab")) 
assert result == ["c", "nn"], f"bad result: {result}"

result = list(without_chars(["a.b.c", "def?", "g#h!i?"], string.punctuation)) 
assert result == ["abc", "def", "ghi"], f"bad result: {result}"

print("Success!!!")

Success!!!


## 4. Generator Warm-up

Write a generator function that yields each word in a string backwards, one at a time. 

The function must **yield** not return.
You may again use a for loop.

In [66]:
def flip_each_word(sentence: str) -> Generator[str, None, None]:
    """
    For a string like: "Hello functional world" this generator yields:
        - olleH
        - lanoitcnuf
        - dlrow
    """
    for word in sentence.split():
        yield word[::-1]

    #Do I need to check if sentence has one word or empty?
    #What would happen if I passed an empty string to this function?


In [67]:
g = flip_each_word("Hello functional world")

In [68]:
# the built in "next" function takes one value at a time from the generator
word = next(g)
assert word == "olleH", f"got {word}"
word = next(g)
assert word == "lanoitcnuf", f"got {word}"
word = next(g)
assert word == "dlrow", f"got {word}"

print("Success!!!!")

Success!!!!


## 5. Generator Pipeline

Write a series of generators to make this pipeline work.

Note that this pipeline does not follow all recommendations given in class,
For instance, we go from str->dict->dict->str->str here to demonstrate some ideas.

In [69]:
def infinite_animals() -> Generator[str, None, None]:
    """ 
    This is an infinite generator.

    The final step in the pipeline will terminate the loop, this function can be left as-is.
    """
    while True:
        yield from animals

In [70]:
def upper_case(inputs: Iterable[str]) -> Generator[str, None, None]:
    """ takes a generator of strings and converts all strings to upper case """
    for item in inputs:
        yield item.upper()

In [71]:
def attach_length(inputs: Iterable[str]) -> Generator[dict, None, None]:
    """ takes a generator of strings and converts each string s to {"data": s, "len": len(s)} """
    for item in inputs:
        yield {"data": item, "len": len(item)}

# What is the point of this function?
#   This demonstrates a common concept of augmenting data during traversal through the pipeline with metadata.
#   In this case, we could have just called len(s) later in the pipeline, but if this was a more expensive calculation.
#   It'd make sense to store it on the object as it traverses the pipeline.
#.  To demonstrate this value, the next function in the pipeline is not allowed to call len.


In [72]:
def threshold_pass(inputs: Iterable[dict], threshold: int) -> Generator[str, None, None]:
    """ 
    Takes an iterable of {"data": s, "len": len(s)} and yields all strings that are longer
    than threshold.

    This method may not call `len` -- see notes on `attach_length` for explanation of what we're mimicking here.
    """
    for item in inputs:
        if item["len"] > threshold:
            yield item["data"]

In [73]:
def run_pipeline(inputs: Iterable[str], length_threshold: int, max_elements: int) -> Generator[str, None, None]:
    """
    This function should run your pipeline, it should take the form:

    for x in f(g(h(...))):
        ...
        yield x
        ...

    Where f/g/h/etc. are the generator functions you just wrote above.

    The function should break out of the loop once max_elements have been processed.

    Parameters:
     - inputs -> iterable of strings to process
     - length_threshold -> threshold for threshold_pass
     - max_elements -> quit after this many elements, even if there are more inputs to process
    """
    for i, x in enumerate(
        upper_case(
            threshold_pass(
                attach_length(inputs),
                length_threshold
        )), start=1):
        yield x
        if i >= max_elements:
            break
#did I have to use infinite_animals() here? I just noticed 
#that I didn't need it. 

In [74]:
result = list(run_pipeline(["abc", "defghi", "jk", "lmnop", "qrs", "tuvwxyz"] , 4, 100)) # process whole list
assert result == ['DEFGHI', 'LMNOP', 'TUVWXYZ'], f"bad result {result}"

result = list(run_pipeline(animals , 4, 2)) # limit to only 2 results
assert result == ['TIGER', 'ZEBRA'], f"bad result {result}"

result = list(run_pipeline(infinite_animals() , 4, 6)) # avoid infinite loop!
assert result == ['TIGER', 'ZEBRA', "HIPPOPOTAMUS"] * 2, f"bad result {result}"

print("Success!!!!!")

Success!!!!!
